In [4]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
from sklearn import linear_model
from scipy import stats
from pandas.core import datetools
%matplotlib inline

In [5]:
new_company_list = [
 'ASGN',
 'AVA',
 'B',
 'BCPC',
 'CACI',
 'CBU',
 'CHE',
 'COLB',
 'FCFS',
 'FFIN',
 'FIVE',
 'GBCI',
 'GDOT',
 'HAE',
 'HI',
 'IART',
 'JBT',
 'KFY',
 'KS',
 'NEOG',
 'NSP',
 'PDCE',
 'PSB',
 'QLYS',
 'SIGI',
 'SKYW',
 'STMP',
 'TREX',
 'VAC',
 'WWE']

In [6]:
month = {'JAN': 1, 'FEB':2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6, 'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT':10, 'NOV': 11, 'DEC': 12}

In [7]:
# Import high frequency data
def import_data(path, company_list):
    path = path               #r'D:\data_raw' # Please put all .csv documents into one folder
    company_list = company_list# Input company name as a list
    company_data = {} # Store company data as a dictionary: {company_name1: data1, company_name2: data2, ...}
    for company in company_list:
        frame = pd.DataFrame()
        list_ = []
        allFiles = glob.glob(path + '/' +company +'_' + '*.csv')
        allFiles.reverse()
        #print (allFiles)
        for file_ in allFiles:
            df = pd.read_csv(file_, index_col=None, header=0, usecols=[0,1], skiprows=[1,])
            list_.append(df)
        frame = pd.concat(list_)
        company_data[company] = frame
        company_data[company].index = pd.RangeIndex(len(company_data[company].index))
    return company_data

In [8]:
company_data = import_data(r'./New_Data', new_company_list)

In [9]:
company_data['ASGN'].head()

,Time Interval,Close
0,05MAR2013_00:00:00.000000,NaN
1,09:30 - 09:35,22.60
2,09:35 - 09:40,22.75
3,09:40 - 09:45,22.75
4,09:45 - 09:50,22.67


In [10]:
#edit time => date time merge big data 

In [11]:
def set_multiindex(company_data, company_list):
    for com in company_list:
        #date_idx = []
        company_data[com]['date'] = pd.Series(np.NaN, index = company_data[com].index)
        date_idx_s = np.array(company_data[com].loc[pd.isnull(company_data[com].Close), ].index)
        date_idx_e = np.append(date_idx_s[1:], company_data[com].shape[0])
        for i, j in zip(date_idx_s, date_idx_e):
            date = company_data[com]['Time Interval'][i][:9]
            d = date[:2]
            m = str(month[date[2:5]])
            y = date[5:]
            trans_date = d+'/'+m+'/'+y
            #date_idx.append(trans_date)
            #company_data[com]['Time Interval'].iloc[i+1:j] = trans_date +' ' + company_data[com]['Time Interval'].iloc[i+1:j]
            company_data[com]['date'].iloc[i+1:j] = pd.to_datetime(trans_date, dayfirst=True)
        company_data[com].set_index(['date', 'Time Interval'], inplace = True)
        company_data[com].dropna(axis = 0, how = 'any', inplace = True)
    return

In [18]:
set_multiindex(company_data, new_company_list[25:])

D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [19]:
for com in new_company_list:
    print (company_data[com].head())

                          Close
date       Time Interval       
2013-03-05 09:30 - 09:35  22.60
           09:35 - 09:40  22.75
           09:40 - 09:45  22.75
           09:45 - 09:50  22.67
           09:50 - 09:55  22.70
                          Close
date       Time Interval       
2013-03-05 09:30 - 09:35  26.21
           09:35 - 09:40  26.30
           09:40 - 09:45  26.41
           09:45 - 09:50  26.36
           09:50 - 09:55  26.39
                          Close
date       Time Interval       
2013-03-05 09:30 - 09:35  26.47
           09:35 - 09:40  26.66
           09:40 - 09:45  26.54
           09:45 - 09:50  26.59
           09:50 - 09:55  26.73
                          Close
date       Time Interval       
2013-03-05 09:30 - 09:35  41.25
           09:35 - 09:40  41.80
           09:40 - 09:45  41.73
           09:45 - 09:50  41.73
           09:55 - 10:00  41.76
                          Close
date       Time Interval       
2013-03-05 09:30 - 09:35  51.67
        

In [20]:
import pickle

In [21]:
with open('processed_raw_data', 'wb') as f:
    pickle.dump(company_data, f)